<a href="https://colab.research.google.com/github/Pedro-Carchi/03MIAR---Algoritmos-de-Optimizacion/blob/main/Trabajo_Pr%C3%A1ctico_Algoritmos_Pedro_Carchi.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Algoritmos de optimización - Trabajo Práctico<br>
Nombre y Apellidos: Pedro Javier Carchi Villalta  <br>
Url: https://github.com/Pedro-Carchi/03MIAR---Algoritmos-de-Optimizacion <br>
Google Colab: https://colab.research.google.com/drive/1t6WQBvOYCgkX7DGzXpxuequrdzF95o-9 <br>
Problema:
>1. Sesiones de doblaje <br>
>2. Organizar los horarios de partidos de una jornada de La Liga<br>
>3. Configuración de Tribunales

Descripción del problema 2 - Organizar los horarios de partidos de una jornada de La Liga:

Desde la La Liga de fútbol profesional se pretende organizar los horarios de los partidos de liga de cada jornada. Se conocen algunos datos que nos deben llevar a diseñar un algoritmo que realice la asignación de los partidos a los horarios de forma que maximice la audiencia.


| Día | Horas    |
|--------|------|
| Viernes    | 20    |
| Sabado    | 12, 16, 18, 20 |
| Domingo  | 12, 16, 18, 20  |
| Lunes   | 20   |


En primer lugar se clasifican los equipos en tres categorías según el numero de seguidores( que tiene relación directa con la audiencia). Hay 3 equipos en la categoría A, 11 equipos de categoría B y 6 equipos de categoría C.

Se conoce estadísticamente la audiencia que genera cada partido según los equipos que se enfrentan y en horario de sábado a las 20h (el mejor en todos los casos)

|  | Categoría A | Categoría B    | Categoría C   |
|--------|------|-----------|-----------|
| Categoría A   | 2 Millones   | 1.3 Millones    | 1 Millones    |
| Categoría B   |    | 0.9 Millones | 0.75 Millones |
| Categoría C   |    |   | 0.47 Millones  |



Si el horario del partido no se realiza a las 20 horas del sábado se sabe que se reduce según los coeficientes de la siguiente tabla

| Hora | Viernes | Sábado | Domingo | Lunes |
|------|---------|--------|---------|-------|
| 12h  |  -      |    0.55    |  0.45   | -  |
| 16h  |  -      |     0.7   |  0.75   | -  |
| 18h  |     -   |     0.8   |  0.85   | -  |
| 20h  |   0.4   |     1   |  1   | 0.40  |


Es posible la coincidencia de horarios pero en este caso la audiencia de cada partido se verá afectada y se estima que se reduce en porcentaje según la siguiente tabla dependiendo del número de coincidencias:

| Coincidencias | -%   |
|---------------|------|
|      0         |   0%   |
|      1         |   25%   |
|      2         |   45%   |
|      3         |   60%   |
|      4         |   70%   |
|      5         |   75%   |
|      6         |   78%   |
|      7         |   80%   |
|      8         |   80%   |

                                        

#Modelo
- ¿Como represento el espacio de soluciones?
- ¿Cual es la función objetivo?
- ¿Como implemento las restricciones?

El problema nos proporciona una jornada de Liga en específico, por lo que la audiencia base de la que partimos es fija. Dependiendo de la hora a la que asignemos el partido (y de si existe otro partido en ese mismo horario) habrá que multiplicar esa audiencia base por un factor de ponderación.


El espacio de soluciones lo podemos representar mediante un vector, cuyo elemento $i$-ésimo corresponde a la hora asignada al partido $i$. Así por ejemplo una posiblle solución es:
$$
[0, 1, 2, 3, 4, 6, 6, 7, 8, 9]
$$
Que significa:
- al partido 0 se asigna el horario 0
- al partido 1 se asigna el horario 1
- al partido 2 se asigna el horario 2
- al partido 3 se asigna el horario 3
- al partido 4 se asigna el horario 4
- al partido 5 se asigna el horario 6
- al partido 6 se asigna el horario 6
- al partido 7 se asigna el horario 7
- al partido 8 se asigna el horario 8
- al partido 9 se asigna el horario 9


La función objetivo se puede expresenar cómo

$$
f = ∑_i A_{i,j}*p_k
$$
con $A_{i,j} = a_i * b_j$, donde $a_i$ es la audiencia base correspondiente al partido $i$, $b_j$ es la ponderación a aplicar debido al asignar el partido al horario $j$, y $p_k$ es la ponderación a aplicar en caso de que existe alguna coincidencia.


#Análisis
- ¿Que complejidad tiene el problema?. Orden de complejidad y Contabilizar el espacio de soluciones

En caso de que no se pudiesen repetir horarios, en total habrían $10! \sim 3.6$ millones combinaciones posibles en el espacio de soluciones.

Si consideramos que cada hora se puede repetir, entonces se trata de un problema de asignación con repetición, lo que se traduce en un total de $10^{10} =  10$ mil millones de combinaciones.

Por tanto, el orden de compejidad del problema es $n!$ en el mejor de los casos. Por lo que resolverlo por el método de fuerza bruta no es viable para $n$ grandes.

#Diseño
- ¿Que técnica utilizo? ¿Por qué?

El problema descrito es muy similar al problema de devolución de cambio, comentado en clase. Por tanto, seguiremos la misma estrategia para resolverlo, es decir un algoritmo voraz.

Un __aspecto fundamental__ a la hora de aplicar este algortimo es tener en cuenta que primero debemos asignar los __partidos con mayor audiencia__ y dejaremos para el final los partidos con menor audiencia. De esta manera nos aseguramos que maximizamos la audiencia.  


El algoritmo consistirá básicamente en lo siguiente:

- ordenar los partidos de la jornada en orden de audiencia, de mayor a menor.

- asignar al partido $i$-ésimo con mayor audiencia, todas las horas disponibles y evaluar la audiencia total, teniendo en cuenta posibles repeticiones.

- nos quedaremos con aquella hora que nos produzca una mayor ganacia de audiencia. Es decir, si en un determinado partido evaluamos que es mejor repetir horario, entonces se repetirá.

- Suponemos (como se dijo en clase) que a lo sumo, podremos repetir 2 veces un mismo horario, es decir que como mucho habrá 1 coincidencia. Para mayores coincidencias, supondremos que no se produce ganancia de audiencia.




Primero definimos los datos proporcionados por el problema. Almacenamos estos datos en diccionarios, ya que nos ayudarán a representar la solución

In [2]:
import numpy as np

In [3]:
# Deinimos los datos proporcionados por el problema

horarios = {
    'V20': 0.4,
    'S12': 0.55,
    'S16': 0.7,
    'S18': 0.8,
    'S20': 1.0,
    'D12': 0.45,
    'D16': 0.75,
    'D18': 0.85,
    'D20': 1.0,
    'L20': 0.4
}

partidos = {
    'Celta-Real Madrid': 1.3,
    'Valencia-R.Sociedad': 1.3,
    'Mallorca-Eibar': 0.47,
    'Athletic-Barcelona': 1.3,
    'Leganes-Osasuna': 0.47,
    'Villareal-Granada': 0.75,
    'Alaves-Levante': 0.9,
    'Espanyol-Sevilla': 0.9,
    'Betis-Valladolid': 0.75,
    'Atletico-Getafe': 0.9
}

resta_repeticion = np.array([0, 0.25, 0.45, 0.60, 0.70, 0.75, 0.78, 0.80, 0.80])
ponderacion_coincidencias = 1 - resta_repeticion


A continuación se presenta el código del algortimo

In [17]:
def asignacion_partidos(partidos, horarios, return_print=False):
  '''
  Args:
    partidos (dict): diccionario cuyas claves son los partidos de la jornada y valores son las audiencias base
    horarios (dict): diccionario cuyas claves son los horarios disponibles y valores son la ponderación al asignar dicho horario
    return_print (bool): si True devuelve impresiones acerca de la resolución del problema

  Returns:
    solucion (array): vector cuyo elemento solucion[i] es la hora a la que se le asigna el partido i
    AUDIENCIA (float): valor de la audiencia máxima
  '''

  audiencia_base = list(partidos.values())
  ponderacion_horarios = list(horarios.values())

  n = len(partidos)
  m = len(horarios)

  # coincidencias de partidos permitidas
  coincidencias = 1
  n_partidos_repetidos = coincidencias + 1

  #inicializar audiencias
  audiencias_asignacion_iesima = [0]*n
  audiencia_final = [0]*n
  audiencias_iniciales = [0]*n

  # inicializar horas y solucion
  horas = [h for h in range(m)]
  conteo_horas = [0]*m
  solucion = [-1]*n

  # ordeno los índices de los partidos desde el que tiene mayor audiencia al que tiene menos
  indices_mejores_partidos = np.argsort(audiencia_base)[::-1]

  # formato del print
  if return_print:
    np.set_printoptions(formatter={'float_kind': lambda x: f"{x:.2f}"})


  for i in indices_mejores_partidos:

    # horarios disponibles, los que no superen el límite de repeticiones
    horarios_disponibles = [h for h in horas if conteo_horas[h]<2]

    if return_print:
      print(f'Asignación partido {i}.  Horarios disponibles: {horarios_disponibles}')
      print('-----------------------')


    for h in horarios_disponibles:

      # Si la hora se repite
      if h in solucion:

        # partido donde se repite la hora
        index  = np.where(np.array(solucion) == h)[0][0] #solo sirve para 2 repeticiones

        # corrijo audiencias para esa hora repetida
        audiencia_provisional = np.copy(audiencias_asignacion_iesima)

        audiencia_provisional[i] = 0.75 * audiencia_base[i]*ponderacion_horarios[h]
        audiencia_provisional[index] = 0.75 * audiencia_base[index]*ponderacion_horarios[h]

        if return_print:
          print(f'Hora {h} (se repite): Audiencia {audiencia_provisional}. Audiencia repitiendo: {sum(audiencia_provisional)} - audiencia sin repetir: {sum(audiencias_iniciales)}')

        # si las audiencias mejoran al repetir horario
        if sum(audiencia_provisional) > sum(audiencias_iniciales):
          indice_hora_max = h # me quedo con la mejor hora
          audiencias_iniciales = np.copy(audiencia_provisional)

      else:
        # si no se repite le asigno su ponderacion de horario sin penalizacion por coincidencia
        audiencias_asignacion_iesima[i] = audiencia_base[i]*ponderacion_horarios[h]

        if return_print:
          print(f'Hora {h}: Audiencia resultante  {np.round(audiencias_asignacion_iesima,2)}. Mejor audiencia hasta el momento {np.round(audiencias_iniciales,2)}')


      # Si la audiencia es mejor, entonces me quedo con esa asignacion
      if sum(audiencias_asignacion_iesima) > sum(audiencias_iniciales):
        indice_hora_max = h # me quedo con la mejor hora
        audiencias_iniciales = np.copy(audiencias_asignacion_iesima)




    # Guardo resultados con la mejor hora encontrada
    audiencia_final[i] = audiencia_base[i]*ponderacion_horarios[indice_hora_max]
    conteo_horas[indice_hora_max] += 1
    solucion[i] = indice_hora_max

    audiencias_asignacion_iesima = np.copy(audiencia_final) # actualizo la asignacion


    if return_print:
      print(f'*** Mejor hora encontrada: {indice_hora_max}.')
      print(f'Solución parcial: {solucion} (-1 = no se ha asignado hora)')
      print(f'Audiencia al terminar asignación de partido {i}:', audiencia_final)
      print()


  AUDIENCIA = sum(audiencia_final)
  if return_print:
    print('****************Solucion Final **********')
    print(solucion)
    print(f'Audiencia_final', AUDIENCIA)


  return solucion, AUDIENCIA





Ejecutamos el algortimo usando como entrada los partidos y los horarios del problema propuesto

In [15]:
SOLUCION, AUDIENCIA = asignacion_partidos(partidos, horarios)

print(f'La solución encontrada es {SOLUCION}, cuya audiencia es {AUDIENCIA}.')
print('O dicho de otro modo:')

for i in range(len(SOLUCION)):
  print(f'Asignar al partido {list(partidos.keys())[i]} el horario {list(horarios.keys())[SOLUCION[i]]}.')


La solución encontrada es [4, 8, 9, 7, 0, 5, 6, 2, 1, 3], cuya audiencia es 6.856.
O dicho de otro modo:
Asignar al partido Celta-Real Madrid el horario S20.
Asignar al partido Valencia-R.Sociedad el horario D20.
Asignar al partido Mallorca-Eibar el horario L20.
Asignar al partido Athletic-Barcelona el horario D18.
Asignar al partido Leganes-Osasuna el horario V20.
Asignar al partido Villareal-Granada el horario D12.
Asignar al partido Alaves-Levante el horario D16.
Asignar al partido Espanyol-Sevilla el horario S16.
Asignar al partido Betis-Valladolid el horario S12.
Asignar al partido Atletico-Getafe el horario S18.


Una manera de comprobar lo que hace internamente el algoritmo, es especificando `return_print=True` como parámetro de entrada.

In [18]:
asignacion_partidos(partidos, horarios, return_print=True)

Asignación partido 0.  Horarios disponibles: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
-----------------------
Hora 0: Audiencia resultante  [0.52 0.00 0.00 0.00 0.00 0.00 0.00 0.00 0.00 0.00]. Mejor audiencia hasta el momento [0 0 0 0 0 0 0 0 0 0]
Hora 1: Audiencia resultante  [0.72 0.00 0.00 0.00 0.00 0.00 0.00 0.00 0.00 0.00]. Mejor audiencia hasta el momento [0.52 0.00 0.00 0.00 0.00 0.00 0.00 0.00 0.00 0.00]
Hora 2: Audiencia resultante  [0.91 0.00 0.00 0.00 0.00 0.00 0.00 0.00 0.00 0.00]. Mejor audiencia hasta el momento [0.72 0.00 0.00 0.00 0.00 0.00 0.00 0.00 0.00 0.00]
Hora 3: Audiencia resultante  [1.04 0.00 0.00 0.00 0.00 0.00 0.00 0.00 0.00 0.00]. Mejor audiencia hasta el momento [0.91 0.00 0.00 0.00 0.00 0.00 0.00 0.00 0.00 0.00]
Hora 4: Audiencia resultante  [1.30 0.00 0.00 0.00 0.00 0.00 0.00 0.00 0.00 0.00]. Mejor audiencia hasta el momento [1.04 0.00 0.00 0.00 0.00 0.00 0.00 0.00 0.00 0.00]
Hora 5: Audiencia resultante  [0.59 0.00 0.00 0.00 0.00 0.00 0.00 0.00 0.00 0.00]. Mejor a

([4, 8, 9, 7, 0, 5, 6, 2, 1, 3], 6.856)

Si nos fijamos en lo anterior, podemos ver que el algoritmo repite horario cada vez que puede. Sin embargo, al hacerlo, el resultado de la audiencia no es mayor que cuando no se repite horario.